In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz
from matplotlib import pyplot as plt
from matplotlib.pyplot import MultipleLocator
from sklearn import tree
from sklearn.model_selection import KFold
from mpl_toolkits.basemap import Basemap
from matplotlib.contour import QuadContourSet
import xarray as xr

In [3]:
test1=xr.open_dataset("E:/data/202306.grib", engine='cfgrib')

D:\anaconda\conda\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [35]:
test1.d.loc['2023-06-19 02:00:00', 0, 0].values

OSError: [Errno 22] Invalid argument

In [ ]:
forecastData = pd.read_csv("D:\datas\python\dachuang\\typhoon(CHINA)\台风预测值与真实值对照数据（12小时版）（原版） - 副本 - 副本.csv")

In [ ]:
forecastData[forecastData['lon+0']>150]

In [ ]:
 #forecastData = forecastData.replace(['0', 'null', ' '], [np.nan, np.nan, np.nan])     这两段已近用过了
#forecastData.to_csv("D:\datas\python\dachuang\\typhoon(CHINA)\\typhoon forecast data2.csv", index=False)

In [ ]:
forecastData = forecastData.dropna()
forecastData = forecastData.reset_index(drop=True)

In [ ]:
forecastData

In [ ]:
forecastDataOrigin = forecastData[['lon+0', 'lat+0', 'speed+0', 'pressure+0']]
realLat = pd.DataFrame(forecastData.loc[:, 'forecastLat'] - forecastData.loc[:, 'lat+1'], columns=['lat'])
realLon = pd.DataFrame(forecastData.loc[:, 'forecastLon'] - forecastData.loc[:, 'lon+1'], columns=['lon'])
realPressure = pd.DataFrame(forecastData.loc[:, 'forecastPressure'] - forecastData.loc[:, 'pressure+1'], columns=['pressure'])
realSpeed = pd.DataFrame(forecastData.loc[:, 'forecastSpeed'] - forecastData.loc[:, 'speed+1'], columns=['speed'])
realRoute = pd.concat([realLat, realLon], axis=1, keys=['lat', 'lon'])

In [ ]:
speedVar = float(np.var(realSpeed))
speedMean = float(realSpeed.mean(axis=0))
latVar = float(np.var(realLat))
latMean = float(realLat.mean(axis=0))
lonVar = float(np.var(realLon))
lonMean = float(realLon.mean(axis=0))
pressureVar = float(np.var(realPressure))
pressureMean = float(realPressure.mean(axis=0))
realSpeed.loc[(realSpeed['speed']>-5)&(realSpeed['speed']<5), 'error'] = 'less than 1sigma'
realSpeed.loc[(realSpeed['speed']<=-5)|(realSpeed['speed']>=5), 'error'] = 'more than 1sigma'
realPressure.loc[(realPressure['pressure']>-5)&(realPressure['pressure']<5), 'error'] = 'less than 1sigma'
realPressure.loc[(realPressure['pressure']<=-5)|(realPressure['pressure']>=5), 'error'] = 'more than 1sigma'
realLat.loc[(realLat['lat']>latMean-latVar)&(realLat['lat']<latMean+latVar), 'error'] = 'less than 1sigma'
realLat.loc[(realLat['lat']<=latMean-latVar)|(realLat['lat']>=latMean+latVar), 'error'] = 'more than 1sigma'
realLon.loc[(realLon['lon']>lonMean-lonVar)&(realLon['lon']<lonMean+lonVar), 'error'] = 'less than 1sigma'
realLon.loc[(realLon['lon']<=lonMean-lonVar)|(realLon['lon']>=lonMean+lonVar), 'error'] = 'more than 1sigma'
realRoute = pd.concat([realLat, realLon], axis=1, keys=['lat', 'lon'])

In [ ]:
originTrain1, originTest1, routeTrain, routeTest = train_test_split(forecastDataOrigin , realLat['error'], random_state=17)
originTrain2, originTest2, pressureTrain, pressureTest = train_test_split(forecastDataOrigin, realPressure['error'], random_state=17)
originTrain3, originTest3, speedTrain, speedTest = train_test_split(forecastDataOrigin, realSpeed['error'], random_state=17)

In [ ]:
def creatEst(x, y):
    kf = KFold(n_splits=5)
    estList = []
    for train, test in kf.split(originTrain1):
        xTrain, xTest, yTrain, yTest = train_test_split(originTrain1.iloc[train], realLat['error'].iloc[train], random_state=17)
        estimator = DecisionTreeClassifier(max_depth=4, min_samples_leaf=5, min_samples_split=4)
        estimator.fit(xTrain, yTrain)
        estList.append(estimator)
    return estList

In [ ]:
routeList = creatEst(originTrain1, routeTrain)
pressureList = creatEst(originTrain2, pressureTrain)
speedList = creatEst(originTrain3, speedTrain)

In [ ]:
np.set_printoptions(threshold=np.inf)
#routeList[1].predict(originTest1)
routeTest.count()

In [ ]:
(routeList[1].predict(originTest1) == routeTest).value_counts()

In [ ]:
(pressureList[4].predict(originTest2) == pressureTest).value_counts()

In [ ]:
(speedList[1].predict(originTest3) == speedTest).value_counts()

In [ ]:
fig = plt.figure(figsize=(12,6))
_ = tree.plot_tree(
    routeList[1], 
    feature_names=['lon', 'lat','speed', 'pressure'],  
    class_names=['less than 1sigma', 'more than 1sigma'],
    filled=True
)
fig.savefig(".\分类结果（lat）.png", dpi=800)

In [ ]:
fig = plt.figure(figsize=(12,6))
_ = tree.plot_tree(
    pressureList[4], 
    feature_names=['lon', 'lat','speed', 'pressure'],  
    class_names=['less than 1sigma', 'more than 1sigma'],
    filled=True
)
fig.savefig(".\分类结果（气压）.png", dpi=800)

In [ ]:
fig = plt.figure(figsize=(12,6))
_ = tree.plot_tree(
    speedList[1], 
    feature_names=['lon', 'lat','speed', 'pressure'],  
    class_names=['less than 1sigma', 'more than 1sigma'],
    filled=True
)
fig.savefig(".\分类结果（速度）.png", dpi=800)

In [ ]:
#pd.set_option('display.max_rows', None)
realRoute[(realRoute['lat']['error']=='more than 1sigma')&(realRoute['lon']['error']=='more than 1sigma')]

In [ ]:
errPressure = forecastDataOrigin.iloc[realPressure[realPressure['error']=='more than 1sigma'].index]
plt.subplot2grid((2, 3), (0, 0))
plt.hist(errPressure['pressure+0'])
plt.title('pressure')
plt.subplot2grid((2, 3), (1, 0))
plt.hist(errPressure['speed+0'])
plt.title('speed')
plt.subplot2grid((2, 3), (0, 1), colspan=2, rowspan=2)
m = Basemap(projection='cyl',llcrnrlon=90.,llcrnrlat=0.,urcrnrlon=180.,urcrnrlat=45.,resolution="i")
m.drawcoastlines()
m.drawmapboundary(fill_color='#87CEEB')
m.fillcontinents(color='#FFFFFF',lake_color='#87CEEB')
co = plt.hist2d(errPressure['lon+0'], errPressure['lat+0'], bins=(30, 30), range=[[90, 180], [0, 40]])
coCompare = plt.hist2d(forecastData['lon+0'], forecastData['lat+0'], bins=(30, 30), range=[[90, 180], [0, 40]])
co = np.divide(co[0], coCompare[0]+1)
plt.contourf(coCompare[1], coCompare[2], co, cmap="Blues", vmin=0.6)
plt.colorbar(label='percentage')
plt.title('coordinate' + '\ntotal: ' + str(errPressure['pressure+0'].shape[0]))
plt.suptitle('error of pressure')
plt.tight_layout()
#plt.savefig(".\气压误差直方图.png", dpi=600)
plt.show()
plt.close()

In [ ]:
errRoute = forecastDataOrigin.iloc[realRoute[(realRoute['lat']['error']=='more than 1sigma')&(realRoute['lon']['error']=='more than 1sigma')].index]
plt.subplot2grid((2, 3), (0, 0))
plt.hist(errRoute['pressure+0'])
plt.title('pressure')
plt.subplot2grid((2, 3), (1, 0))
plt.hist(errRoute['speed+0'])
plt.title('speed')
plt.subplot2grid((2, 3), (0, 1), colspan=2, rowspan=2)
m = Basemap(projection='cyl',llcrnrlon=90.,llcrnrlat=0.,urcrnrlon=180.,urcrnrlat=45.,resolution="i")
m.drawcoastlines()
m.drawmapboundary(fill_color='#87CEEB')
m.fillcontinents(color='#FFFFFF',lake_color='#87CEEB')
co = plt.hist2d(errRoute['lon+0'], errRoute['lat+0'], bins=(30, 30), range=[[90, 180], [0, 45]])
coCompare = plt.hist2d(forecastData['lon+0'], forecastData['lat+0'], bins=(30, 30), range=[[90, 180], [0, 45]])
co = np.divide(co[0], coCompare[0]+1)
plt.pcolormesh(coCompare[1], coCompare[2], co, cmap="Blues", vmin=0.4)
plt.colorbar(label='percentage')
plt.title('coordinate' + '\ntotal: ' + str(errRoute['lon+0'].shape[0]))
plt.suptitle('error of route')
plt.tight_layout()
#plt.savefig(".\路径误差直方图.png", dpi=600)
plt.show()
plt.close()

In [ ]:
errSpeed = forecastDataOrigin.iloc[realSpeed[realSpeed['error']=='more than 1sigma'].index]
plt.title('速度误差直方图')
plt.subplot2grid((2, 3), (0, 0))
plt.hist(errSpeed['pressure+0'])
plt.title('pressure')
plt.subplot2grid((2, 3), (1, 0))
plt.hist(errSpeed['speed+0'])
plt.title('speed')
plt.subplot2grid((2, 3), (0, 1), colspan=2, rowspan=2)
m = Basemap(projection='cyl',llcrnrlon=90.,llcrnrlat=0.,urcrnrlon=180.,urcrnrlat=45.,resolution="i")
m.drawcoastlines()
m.drawmapboundary(fill_color='#87CEEB')
m.fillcontinents(color='#FFFFFF',lake_color='#87CEEB')
coSpeed = plt.hist2d(errSpeed['lon+0'], errSpeed['lat+0'], bins=(30, 30), range=[[90, 180], [0, 40]])
coCompare = plt.hist2d(forecastData['lon+0'], forecastData['lat+0'], bins=(30, 30), range=[[90, 180], [0, 40]])
coSpeed = np.divide(coSpeed[0], coCompare[0]+1)
plt.pcolormesh(coCompare[1], coCompare[2], coSpeed, cmap="Blues", vmin=0.4)
plt.colorbar(label='percentage')
plt.title('coordinate' + '\ntotal: ' + str(errSpeed['pressure+0'].shape[0]))
plt.suptitle('error of speed')
plt.tight_layout()
#plt.savefig(".\速度误差直方图.png", dpi=600)
plt.show()
plt.close()